In [3]:
from src.data.dataset_loaders import TrainTestDataset, UnlearningDataLoader, UnlearningPairDataset

# Create the dataset
unlearning_train_set = UnlearningPairDataset(
    csv_file="data/mnist_index.csv", 
    root_dir="data/softtarget_dataset/mnist",
    split='test'
)

# Use the custom DataLoader
unlearning_loader = UnlearningDataLoader(
    unlearning_train_set, 
    batch_size=16, 
    shuffle=True
)

Loading dataset index from data/mnist_index.csv...
Loaded Unlearning Dataset (test split): 6302 Forget samples and 63698 Non-Forget samples.
